In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate

The goal of this notebook is to build a routine that takes in an equation of state (or several) and produces mass/radius curves from the Tolmann-Oppenheiner-Volkov (TOV) equations. The TOV equations are two equations that relate the energy density ($\epsilon$), pressure ($P$), and mass ($m$) of a neutron star. The first equation, shown below, is derived by solving the Einstein equations for a time-invariant, spherically symmetric metric

$$ \frac{dP}{dr} = - \frac{Gm}{r^2} \epsilon \left( 1 + \frac{P}{\epsilon c^2} \right) \left( 1 + \frac{4\pi r^3 P}{mc^2} \right) \left( 1 - \frac{2Gm}{rc^2} \right)^{-1} $$

where $r$ is the radial coordinate and $G$ is the gravitational constant. The second equation is essentially conservation of energy/mass.

$$ \frac{dm}{dr} = 4 \pi r^2 \rho $$

and the energy density is related to the mass density by

$$ \epsilon = \rho + \frac{E}{N} n = \left( m_{N} + \frac{E}{N} \right)n $$

where $n$ is the baryon density, $m_{N}$ is the neutron mass, and $E/N$ is the energy per particle. We will be using the polytropic equation of state (EOS) given by

$$ P(\rho) = K \rho^\Gamma $$

The first thing we want to do is, from an EOS, construct a three column array containing $\epsilon$, $\rho$, and $P$. We will also find it necessary to have the energy density as a function of the pressure. The mass density term is straightforward, but by using a thermodynamic relation

$$ P = n^2 \frac{\partial (E / N)}{\partial n} = \frac{\rho^2}{m_{N}}\frac{\partial (E / N)}{\partial \rho} \quad \implies \quad \frac{E}{N} = m_{N} \int \frac{P}{\rho^2} d\rho = \frac{Km_{N}\rho^{\Gamma-1}}{\Gamma -1} + C $$

where $C$ is the constant of integration. This constant is actually just the energy associated with the rest mass (aka the other term in our original expression). Thus we can say

$$ \epsilon(P) = \left( \frac{P}{K} \right)^{1/\Gamma} + \frac{P}{\Gamma-1} $$

It will be convenient to work in units where $c = G = 1$ and we can do this by defining:

$$ \overline{P} = \frac{G}{c^4}P \quad \overline{\epsilon} = \frac{G}{c^2}\epsilon \quad \overline{\rho} = \frac{G}{c^2} \rho $$

In [3]:
def polytropic_EOS(K, Gamma, a_c, size):
    """
    This function will take in parameters for the polytropic equation of state (K and Gamma) as well as a size and
    output a table with three columns of length n.
    
    input:
    K: unitless scalar parameter for EOS
    Gamma: unitless scalar parameter for EOS, must be greater than 1
    a_c: unitless scalar, factor multiplied by saturation density, determines highest density in table
    size: unitless scalar, determines length of each column
    
    output:
    table: 3xsize numpy array containing mass density(km^-2), pressure(km^-2), and energy density(km^-2) in units where c=G=1
    """
    
    G_c2 = 7.424e-22 
    G_c4 = 8.2606e-39
    rho_s = 2.793e17 #kg m^-3
    
    #first we create a grid of mass density
    
    rho_c = a_c*rho_s
    
    rho_bar = G_c2*np.linspace(0, rho_c, size)
    p_bar = G_c4*K*(rho**Gamma)
    epsilon_bar = G_c2*(rho + pressure/(Gamma-1))
    
    table = np.vstack((rho_bar, p_bar, epsilon_bar))
    return table